In [28]:
import fitz
import os
from pathlib import Path
import spacy
import pprint

In [56]:
def get_centroid(x0, x1, y0, y1):
    pass

In [2]:
def extrair_texto_por_fatias_pagina_unica(caminho_pdf, num_fatias):
    """
    Extrai texto da PRIMEIRA página de um PDF, dividindo-a 
    em N fatias horizontais.
    
    Argumentos:
    caminho_pdf -- O caminho para o arquivo PDF.
    num_fatias -- O número de "fatias" horizontais desejadas.

    Retorna:
    Um dicionário, onde cada chave é a fatia e o valor é o texto.
    Ex: {"fatia_1": "texto da fatia 1", "fatia_2": "texto da fatia 2"}
    """
    
    try:
        doc = fitz.open(caminho_pdf)
    except Exception as e:
        print(f"Erro ao abrir o PDF {caminho_pdf}: {e}")
        return {} # Retorna um dicionário vazio em caso de erro

    textos_das_fatias_dict = {}
    
    pagina = doc[0] 
    texto = pagina.get_text()

    doc.close()
    
    # Retorna o dicionário preenchido
    return textos_das_fatias_dict, doctext

In [3]:
nlp = spacy.load("pt_core_news_lg")

In [10]:
caminho_do_arquivo = Path("./files/tela_sistema_1.pdf")

### Extraindo todas as frases

In [62]:
#Extraindo texto do arquivo
doc = fitz.open(caminho_do_arquivo)
page = doc.load_page(0)  # Load the first page
        
text = page.get_text("text")

text = text.split("\n")

In [64]:
text

['Consulta de Cobrança',
 'Detalhamento de saldos por parcelas:',
 'Data Referência: 05/09/2025',
 '<< Esconder Filtros',
 'Data Referência:',
 'Seleção de parcelas:',
 'Intervalo de parcel5:',
 'Atualizar',
 '03/09/2025',
 'Vencidas',
 'Vcto mais antigo',
 'Dias atraso Sistema',
 'Saldo Vencido',
 'Saldo a Vencer',
 'Total Geral',
 '12/10/2025',
 '0 CONSIGNADO',
 '0,00',
 '76.871,20',
 '76.871,20',
 'Total:',
 '76.871,20',
 '']

### Pegando as posições das frases

In [55]:
data = page.get_text("dict")

data["blocks"][2]["lines"][2]["spans"][0]

{'size': 11.0,
 'flags': 0,
 'bidi': 0,
 'char_flags': 16,
 'font': 'Helvetica',
 'color': 0,
 'alpha': 0,
 'ascender': 1.0750000476837158,
 'descender': -0.29899999499320984,
 'text': '<< Esconder Filtros',
 'origin': (637.0, 193.82501220703125),
 'bbox': (637.0, 182.00001525878906, 732.3701782226562, 197.114013671875)}

In [57]:
texts = []

In [66]:
data

{'width': 744.0,
 'height': 591.0,
 'blocks': [{'number': 0,
   'type': 1,
   'bbox': (0.0, 0.0, 744.0, 591.0),
   'width': 992,
   'height': 788,
   'ext': 'png',
   'colorspace': 3,
   'xres': 96,
   'yres': 96,
   'bpc': 8,
   'transform': (744.0, 0.0, -0.0, 591.0, 0.0, 0.0),
   'size': 65522,
   'image': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xe0\x00\x00\x03\x14\x08\x02\x00\x00\x00\xb3F\xda%\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00\xff\xa4IDATx\x9c\xec\xbd\x87w\x14\xd7\xb6\xee{\xff\x97\xbb\xf7\x19/\xdds\xdf}\xf7\x9d\xfb\xdex\xfb\x9csm\xc0\xd8\x18\x07\x9c#68`\x9b\xed\x88\xbdm\xb3q\xf6\xc69g\x1bl!\x10A$\t!!\t\x05\x14@\x11%$$P\xce9u\xe5\xee\xean\xbdo\xae\xee\x96Z\x11\x81}\x0e\xb5\x8f\xbf\xdf\x98CCjU\xadZ\xa9\xe6\xfa\xe6\xaaU\xab\xff\xd3?(\xfe\xf0\x87?\xdc\xbd\xea\xff\xf2\x1dX\xaf\x1f\xdb@\xa3\xd1\xfe\x0e,\xed\x11=\xfd\t\xe3\xc4\x9fgY\xc6S\xfa\xf1\xc7\xf4c\x1b\xafF\x96\x1eV\xb6AO\xddhd>\xefo\xcd\x0c\x8e^\x08\x8e\xb68\xe7\x8f\x98\xb9\x7f\xd5\

In [59]:
for  i,block in enumerate(data["blocks"]):
    if data["blocks"][i]["type"] == 0:
        for j, line in enumerate(data["blocks"][i]["lines"]):
            text = data["blocks"][i]["lines"][j]["spans"][0]["text"]
            texts.append(text)

In [65]:
texts

['Consulta de Cobrança',
 'Detalhamento de saldos por parcelas:',
 'Data Referência: 05/09/2025',
 '<< Esconder Filtros',
 'Data Referência:',
 'Seleção de parcelas:',
 'Intervalo de parcel',
 'Atualizar',
 '03/09/2025',
 'Vencidas',
 'Vcto mais antigo',
 'Dias atraso Sistema',
 'Saldo Vencido',
 'Saldo a Vencer',
 'Total Geral',
 '12/10/2025',
 '0 CONSIGNADO',
 '0,00',
 '76.871,20',
 '76.871,20',
 'Total:',
 '76.871,20']